In [1]:
from gevent import monkey
monkey.patch_all(thread=False)

from gevent.pool import Pool
pool = Pool(5)

from gevent.lock import BoundedSemaphore
semaphore_correct, semaphore_error, semaphore_remain = BoundedSemaphore(1), BoundedSemaphore(1), BoundedSemaphore(1)

from IPython.display import clear_output
from datetime import datetime

import os
import shutil
directory_out = './file/3.part'
if os.path.exists(directory_out):
    shutil.rmtree(directory_out)
os.makedirs(directory_out)
import re

# = = =  = = =  = = =

import pandas as pd

# = = =  = = =  = = =

import requests

import sys
sys.path.append('../00.Tools')
from crawler_configuration import get_header, get_proxy

# = = =  = = =  = = =

from bs4 import BeautifulSoup
from lxml import etree

# = = =  = = =  = = =

import json

# = = =  = = =  = = =

directory_in = './file/2.url'
list_file = [file for file in sorted(list(os.walk(directory_in))[0][2]) if file not in ['.DS_Store'] and os.path.isfile(os.path.join(directory_in, file))]
for file in list_file:

    # = = =  = = =  = = =

    print('Data Loading...')
    list_dict_correct, list_series_error, df_input = [], [], pd.read_excel(os.path.join(directory_in, file),
                                                                           header=0,
                                                                           dtype=str).fillna('')
    print('Loading Done !')
    print()

    df_input['No'] = df_input['No'].astype(int)

    crawler_remain = len(df_input)

    print(f'总数量：{crawler_remain}')
    print()

    # = = =  = = =  = = =

    def crawler(crawler_tuple):
        global list_dict_correct, list_series_error, crawler_remain

        crawler_status, (crawler_index, crawler_series) = False, crawler_tuple

        # = = =  = = =  = = =

        try:
            crawler_retry = 0
            while True:
                crawler_retry += 1

                try:
                    url_request = crawler_series['Url']

                    resp = requests.get(url_request,
                                        headers=get_header(),
                                        proxies=get_proxy(),
                                        timeout=(5, 15))

                    if resp.status_code == 200:
                        break
                except KeyboardInterrupt:
                    break
                except:
                    continue

            # = = =  = = =  = = =

            soup = BeautifulSoup(resp.text, 'lxml')
            html = etree.HTML(str(soup))

            # = = =  = = =  = = =

            dict_ = json.loads(resp.text.split('$MC=(window.$MC||[]).concat(')[1].split(']}]})</script>')[0] + ']}]}')

            # = = =  = = =  = = =

            if crawler_series['Url'].startswith('https://www.ebay.com/'):
                market, country, language, locale = 'EBAY-US', 'US', 'en', 'en-US'
            elif crawler_series['Url'].startswith('https://www.ebay.de/'):
                market, country, language, locale = 'EBAY-DE', 'DE', 'de', 'de-DE'
            elif crawler_series['Url'].startswith('https://www.ebay.co.uk/'):
                market, country, language, locale = 'EBAY-GB', 'GB', 'en', 'en-GB'
            elif crawler_series['Url'].startswith('https://www.ebay.com.au/'):
                market, country, language, locale = 'EBAY-AU', 'AU', 'en', 'en-AU'
            elif crawler_series['Url'].startswith('https://www.ebay.ca/'):
                market, country, language, locale = 'EBAY-ENCA', 'CA', 'en', 'en-CA'
            else:
                market, country, language, locale = 'fuck', 'fuck', 'fuck', 'fuck'

            # = = =  = = =  = = =

            if market == 'fuck' and country == 'fuck' and language == 'fuck' and locale == 'fuck':
                pass
            else:
                if dict_['o']['g']['itemId'].strip() != crawler_series['Item Number'] or dict_['o']['g']['marketplaceId'].strip() != market or dict_['o']['g']['country'].strip() != country or dict_['o']['g']['language'].strip() != language or dict_['o']['g']['locale'].strip() != locale:
                    raise

            # = = =  = = =  = = =

            list_dict = [list_[2] for list_ in dict_['o']['w'] if 'model' in list_[2]]

            # = = =  = = =  = = =

            dict_ = list_dict[0]['model']['modules']['COMPATIBILITY_TABLE']['paginatedTable']['pagination']['pages'][-1] if 'COMPATIBILITY_TABLE' in list_dict[0]['model']['modules'] else {}
            dict_compatibility = {'page': int(dict_['text'].strip()),
                                  'market_place_id': dict_['action']['params']['marketplaceId'],
                                  'item_id': dict_['action']['params']['itemId'],
                                  'seller_name': dict_['action']['params']['sellerName'],
                                  'category_id': dict_['action']['params']['categoryId'],
                                  'api': dict_['action']['URL'].strip()} if dict_ else {}

            # = = =  = = =  = = =

            dict_key, dict_item = {}, list_dict[0]['model']['modules']['ABOUT_THIS_ITEM']['sections']['features']['dataItems'] if 'ABOUT_THIS_ITEM' in list_dict[0]['model']['modules'] else {}
            for value in dict_item.values():
                key = value['labels'][0]['textSpans'][0]['text'].strip()
                if key in dict_key:
                    dict_key[key] += 1
                else:
                    dict_key[key] = 1

            dict_specific, dict_count = {}, {}
            for i, value in enumerate(dict_item.values()):
                key = value['labels'][0]['textSpans'][0]['text'].strip()
                if dict_key[key] == 1:
                    dict_specific[str(i)] = {key: value['values'][0]['textualDisplays'][0]['textSpans'][0]['text'].strip() if 'textualDisplays' in value['values'][0] else value['values'][0]['textSpans'][0]['text'].strip()}
                else:
                    if key in dict_count:
                        dict_count[key] += 1
                    else:
                         dict_count[key] = 1
                    dict_specific[str(i)] = {f'{key}-{dict_count[key]}': value['values'][0]['textualDisplays'][0]['textSpans'][0]['text'].strip() if 'textualDisplays' in value['values'][0] else value['values'][0]['textSpans'][0]['text'].strip()}

            # = = =  = = =  = = =

            description_url = list_dict[0]['model']['modules']['ITEM_DESCRIPTION_MIN_VIEW_MODULE']['sections'][0]['dataItems'][0]['action']['URL'].strip() if 'dataItems' in list_dict[0]['model']['modules']['ITEM_DESCRIPTION_MIN_VIEW_MODULE']['sections'][0] else list_dict[0]['model']['modules']['ITEM_DESCRIPTION_MIN_VIEW_MODULE']['sections'][0]['action']['URL'].strip()

            # = = =  = = =  = = =

            title = list_dict[0]['model']['modules']['TITLE']['mainTitle']['textSpans'][0]['text'].strip()

            # = = =  = = =  = = =

            subtitle = list_dict[0]['model']['modules']['TITLE']['subTitle']['textSpans'][0]['text'].strip() if 'subTitle' in list_dict[0]['model']['modules']['TITLE'] else ''

            # = = =  = = =  = = =

            if 'binModel' in list_dict[0]['model']['modules']['BUY_BOX']:
                price = list_dict[0]['model']['modules']['BUY_BOX']['binModel']['price']['textSpans'][0]['text'].strip()
            elif 'bidPrice' in list_dict[0]['model']['modules']['BUY_BOX']:
                price = list_dict[0]['model']['modules']['BUY_BOX']['bidPrice']['textSpans'][0]['text'].strip()
            else:
                price = list_dict[0]['modules']['BUY_BOX']['bidPrice']['textSpans'][0]['text'].strip()

            # = = =  = = =  = = =

            dict_quantity, list_quantity = {}, list_dict[0]['model']['modules']['QUANTITY']['availabilitySignal']['textSpans'] if 'QUANTITY' in list_dict[0]['model']['modules'] else []
            for i, quantity in enumerate(list_quantity):
                dict_quantity[str(i)] = quantity['text'].strip()

            # = = =  = = =  = = =

            dict_price, list_dict_price = {}, list_dict[0]['model']['modules']['VOLUME_PRICING']['group']['entries'] if 'VOLUME_PRICING' in list_dict[0]['model']['modules'] else []
            for i, dict_ in enumerate(list_dict_price):
                dict_price[str(i)] = dict_['secondaryLabel']['accessibilityText'].strip()

            # = = =  = = =  = = =

            dict_src, list_media = {}, list_dict[0]['model']['modules']['PICTURE']['mediaList'] if 'mediaList' in list_dict[0]['model']['modules']['PICTURE'] else []
            for i, media in enumerate(list_media):
                dict_src[str(i)] = media['image']['originalImg']['URL'].strip()

            # = = =  = = =  = = =

            store_name, list_section = '', list_dict[0]['model']['modules']['STORE_INFORMATION']['sections']
            for section in list_section:
                if 'logo' in section and 'title' in section['logo']:
                    store_name = section['logo']['title']
                    break

            # = = =  = = =  = = =

            store_url, list_section = '', list_dict[0]['model']['modules']['STORE_INFORMATION']['sections']
            for section in list_section:
                if 'logo' in section and 'action' in section['logo'] and 'URL' in section['logo']['action']:
                    store_url = section['logo']['action']['URL'].split('?')[0]
                    break

            # = = =  = = =  = = =

            with semaphore_correct:
                list_dict_correct.append({'No': crawler_series['No'],
                                          'Item Number': crawler_series['Item Number'],
                                          'Title': title,
                                          'Price': price,
                                          'Picture': '',
                                          'Url': crawler_series['Url'],
                                          'eBay Market': country,
                                          'Subtitle': subtitle,
                                          'Store Name': store_name,
                                          'Store Url': store_url,
                                          'Json_Src': json.dumps(dict_src),
                                          'Description Url': description_url,
                                          'Json_Quantity': json.dumps(dict_quantity),
                                          'Json_Price': json.dumps(dict_price),
                                          'Json_Specific': json.dumps(dict_specific),
                                          'Json_Compatibility': json.dumps(dict_compatibility)})

            # = = =  = = =  = = =

            crawler_status = True
        except:
            crawler_series['Request_Url'] = url_request

            with semaphore_error:
                list_series_error.append(crawler_series)

        # = = =  = = =  = = =

        if list_file.index(file) % 3 == 0 and list_file.index(file) != 0 and crawler_remain == 1 or crawler_remain % 30 == 0:
            clear_output()
        print(f'''[{crawler_status}] - [请求{crawler_retry}次] - [剩余{crawler_remain - 1}条] - [{file.removesuffix('.xlsx')}（{(list_file.index(file) + 1) / len(list_file) * 100:.2f}%）] - [{datetime.now().strftime('%Y/%m/%d %H:%M:%S')}] - {crawler_series['No']}. {url_request}\n''')

        with semaphore_remain:
            crawler_remain -= 1

    # = = =  = = =  = = =

    pool.map(crawler, df_input.iterrows())

    print('Data Outputting...')
    if list_dict_correct:
        df_correct = pd.DataFrame(list_dict_correct).sort_values(by=['No'],
                                                                 ascending=[True],
                                                                 ignore_index=True)
        df_correct.to_excel(os.path.join(directory_out, f'''{file.removesuffix('.xlsx')}-{datetime.now().strftime('%Y%m%d_%H%M%S')}.xlsx'''), index=False)
    print('Output Finished !')
    print()
    if list_series_error:
        df_error = pd.DataFrame(list_series_error).sort_values(by=['No'],
                                                               ascending=[True],
                                                               ignore_index=True)
        df_error.to_excel(f'''./file/{file.removesuffix('.xlsx')}-part_error.xlsx''', index=False)
        print('- Error Occurred -')
        print()

# = = =  = = =  = = =

list_file = [file for file in sorted(list(os.walk(directory_out))[0][2]) if file not in ['.DS_Store'] and os.path.isfile(os.path.join(directory_out, file))]
for file in list_file:
    os.rename(os.path.join(directory_out, file), os.path.join(directory_out, re.sub(r'-[0-9]{8}_[0-9]{6}.xlsx$', '.xlsx', file)))

# = = =  = = =  = = =

print('Done ~')

Data Loading...
Loading Done !

总数量：1

[True] - [请求1次] - [剩余0条] - [crawler_2-part_error-part_error（100.00%）] - [2024/12/06 15:16:26] - 48. https://www.ebay.com/itm/286138807603?_ul=US&_stpos=91710&orig_cvip=true

Data Outputting...
Output Finished !

Done ~
